In [1]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2022-12-29 15:45:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.237.133.81, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  --.-KB/s    in 0.1s    

2022-12-29 15:45:16 (102 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip


In [2]:
!/ngrok authtoken 2HhuvFCdaGTp6lQIJhj6N3rbGyg_4LK4Tsv72PsmMCwvzRAra

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
!pip install flask_ngrok
!pip install line-bot-sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
data = pd.read_csv("車輛資料（最終成果）.csv",encoding="utf-8")

In [6]:

data.loc[[6,1086,1087,1154,1155,1156,1157],"價格"] = 0

In [7]:
data["價格"] = data["價格"].astype("float64")*10000
data = data.drop([6,1086,1087,1154,1155,1156,1157])

In [8]:
import requests
from bs4 import BeautifulSoup

def news(brand):
  url = "https://tw.news.yahoo.com/search?p="+brand+"&fr=news&fr2=p%3Anews%2Cm%3Asb"
  req = requests.get(url)
  soup = BeautifulSoup(req.text, "html.parser")
  url = "https://tw.news.yahoo.com" + soup.select(".Mb(5px) .Fw(b)")[0]["href"]
  return url

def bonus():
  url = "https://yes-news.com/yespick/401494/%E5%A5%BD%E4%B9%85%E6%B2%92%E8%B7%9F%E5%A4%A7%E5%AE%B6%E5%88%86%E4%BA%AB%E5%AD%B8%E7%94%9F%E5%A6%B9%E5%AD%90%E5%95%A6%E4%BB%8A%E5%A4%A9%E9%80%99%E4%BD%8D%E6%98%AF%E5%B0%B1%E8%AE%80%E8%BC%94%E4%BB%81%E5%A4%A7%E5%AD%B8%E6%9C%83%E8%A8%88%E7%B3%BB%E7%9A%84%E6%AD%A3%E5%A6%B9%E4%BD%99%E5%AE%B6%E5%84%80"
  req = requests.get(url)
  soup = BeautifulSoup(req.text, "html.parser")
  img = random.choice(soup.select("p img"))["src"]
  return img

In [ ]:
from flask import Flask, request
import requests, json, time
from flask_ngrok import run_with_ngrok
import openai
import requests
from bs4 import BeautifulSoup
import random
app = Flask(__name__)

@app.route("/")
def home():
    return "<h1>hello world</h1>"

@app.route('/webhook', methods=['POST'])
def webhook():
    req = request.get_json()
    print(req)
    queryText = req['queryResult']["queryText"]
    reText = req['queryResult']['fulfillmentText']          # 取得 Dialogflow 的回應文字
    intent = req['queryResult']['intent']['displayName']    # 取得 intent 分類
    replytoken = req['originalDetectIntentRequest']['payload']['data']['replyToken']  # 取得 LINE replyToken
    token = 'YOUR_YOKEN'

    # 如果收到的 intent 是看車
    if intent == '看車':   
      if req['queryResult']["parameters"]["energy"] and req['queryResult']["parameters"]["number"] and req['queryResult']["parameters"]["brand"]:

          brand = req['queryResult']["parameters"]["brand"][0]
          price = req['queryResult']["parameters"]["number"][0]
          energy = req['queryResult']["parameters"]["energy"][0]

          if not price:
            price = 5000000000000000
          
          #價錢區間
          if len(req['queryResult']["parameters"]["number"]) == 2:
            price = req['queryResult']["parameters"]["number"][1]
          #價錢以上
          if req['queryResult']["parameters"]["up"]:
            if energy == "不拘"  and brand != "不拘":
              query = data[data["車"].str.contains(brand)][data["價格"] >= price]
            
            elif energy != "不拘"  and brand == "不拘":
              query = data[data["動力來源"].str.contains(energy)][data["價格"] >= price]
            
            elif energy == "不拘"  and brand == "不拘":
              query = data[data["價格"] >= price]  
            else:
              query = data[data["車"].str.contains(brand)][data["動力來源"].str.contains(energy)][data["價格"] >= price]
          else:          
          #正常查詢              
            if energy == "不拘"  and brand != "不拘":
              query = data[data["車"].str.contains(brand)][data["價格"] <= price]
              
            elif energy != "不拘"  and brand == "不拘":
              query = data[data["動力來源"].str.contains(energy)][data["價格"] <= price]
              
            elif energy == "不拘"  and brand == "不拘":
              query = data[data["價格"] <= price]

            else:
              query = data[data["車"].str.contains(brand)][data["動力來源"].str.contains(energy)][data["價格"] <= price]

          #有茶道 
          try:
            if req['queryResult']["parameters"]["up"]:
              car = query.sort_values("價格",ascending=[True]).head(1)
            else:
              car = query.sort_values("價格",ascending=[False]).head(1)
            a = "車款: " + car["車"].values[0]
            b = "版本: " + car["版本"].values[0]
            c = "動力來源: " + car["動力來源"].values[0]
            d = "特色: " + car["特色"].values[0]
            e = "價格: " + str(car["價格"].values[0])
              
            url = car["網址"].values[0]
            img = car["照片網址"].values[0]

            text = a + "\n" + b + "\n" + c + "\n" + d+ "\n" + e
            
            
            headers = {'Authorization':'Bearer ' + token,'Content-Type':'application/json'}
            body = {
                    'replyToken':replytoken,
                    'messages':[
                          
                          {
                            "type": "text",
                            "text": text
                          },
                          {
                              "type":"image",
                              'originalContentUrl': img,
                              'previewImageUrl': img
                          },
                          {
                              "type":"text",
                              "text":url
                          },
                          {
                              "type":"text",
                              "text":"寶貝我很厲害吧~快誇獎我💕"
                          },
                          {
                              "type":"text",
                              "text":reText + "💕"
                          }
                        
                        ]
                      
                    }
                # 使用 requests 方法回傳訊息到 ＬINE
            result = requests.request('POST', 'https://api.line.me/v2/bot/message/reply',headers=headers,data=json.dumps(body).encode('utf-8'))
            print(result.text)
                # 完成後回傳訊息到 Dialogflow
            return {
                    
                    "source": "webhookdata"
              }
          #沒查到
          except:
            headers = {'Authorization':'Bearer ' + token,'Content-Type':'application/json'}
            body = {
                    'replyToken':replytoken,
                    
                    
                    'messages':[
                          
                          {
                            "type": "text",
                            "text": "寶貝台灣最近好像沒這種車，要不要找找其他的呢💕"
                          }
                        
                        ]
                    }
                # 使用 requests 方法回傳訊息到 ＬINE
            result = requests.request('POST', 'https://api.line.me/v2/bot/message/reply',headers=headers,data=json.dumps(body).encode('utf-8'))
            print(result.text)
                # 完成後回傳訊息到 Dialogflow
            return {
                    
                    "source": "webhookdata"
              }              
    if intent == "看新聞":
      brand = req['queryResult']["parameters"]["brand"]
      if brand == "不拘":
        brand = random.choice("""Alfa-Romeo Aston-Martin Audi Bentley BMW Bugatti Buick Cadillac Chery Chrysler Citroen Daihatsu Ferrari Fiat Ford Honda Hyundai Infiniti Jaguar Jeep Kia Lamborghini Land-Rover
                      Lexus Lotus Luxgen M-Benz Mahindra Maserati Mazda McLaren MG Mini Mitsubishi Morgan Nissan Opel Peugeot Porsche Proton Renault Rolls-Royce 
                      Saab Skoda Smart Ssangyong Subaru Suzuki 
                      Tesla Tobe Toyota Volkswagen Volvo """.split())

      url = news(brand)
    
      return {
            "fulfillmentText": url
        }
        
    #例外處理
    if intent == "Default Fallback Intent":
        #抽抽小姐姐
        if queryText == "抽":
          headers = {'Authorization':'Bearer ' + token,'Content-Type':'application/json'}
          img = bonus()
          body = {
                    'replyToken':replytoken,
                    'messages':[
                          
                          
                          {
                              "type":"image",
                              'originalContentUrl': img,
                              'previewImageUrl': img
                          }
                        
                        ],
                      
                    }
                
          result = requests.request('POST', 'https://api.line.me/v2/bot/message/reply',headers=headers,data=json.dumps(body).encode('utf-8'))
          
                
          return {
                    
                    "source": "webhookdata"
              }
        #gpt3
        else:
          
          openai.api_key = "OPEN_AI_API_KEY"
          response = openai.Completion.create(
            model="text-davinci-003",
            prompt=queryText,
            temperature=0.9,
            max_tokens=150,
            top_p=1,
            frequency_penalty=0.0,
            presence_penalty=0.6,
            stop=[" Human:", " AI:"]
          )
          
          response = response["choices"][0]["text"].replace('\n', '').replace("您","寶貝").replace("你","寶貝")
          if "。" in response:
            response = response.split("。")[0]
          return {
              "fulfillmentText": response
          }

run_with_ngrok(app)     # 啟用 ngrok
app.run()